## Descripcion

## Load Data

In [47]:
import pandas as pd

In [54]:
# Importa titles
All = pd.read_parquet('processed_data/titles.parquet')
# All = pd.read_csv("data/all.csv")

In [55]:
Score = pd.read_parquet('processed_data/score.parquet')
# Score_CSV = pd.read_csv('ignore/score.csv')

In [50]:
Score.tail(3)

,userId,movieId,score,platform
11024286,124380,hs305,3,hulu
11024287,124380,ns7881,4,netflix
11024288,124380,as883,1,amazon


In [27]:
# Obtiene informacion
print('All platforms')
print('Duplicates:',All.duplicated().any())
print('Files, Columns:',All.shape)
print(All.isnull().sum())
All.tail(2)

All platforms
Duplicates: False
Files, Columns: (22998, 13)
id                   0
platform             0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added        9554
release_year         0
duration_int         0
duration_type      482
listed_in            0
description          4
dtype: int64


,id,platform,type,title,director,cast,country,date_added,release_year,duration_int,duration_type,listed_in,description
22996,ns8806,netflix,movie,zoom,peter hewitt,"tim allen, courteney cox, chevy chase, kate ma...",united states,2020-01-11,2006,88,min,"children & family movies, comedies","dragged from civilian life, a former superhero..."
22997,ns8807,netflix,movie,zubaan,mozez singh,"vicky kaushal, sarah-jane dias, raaghav chanan...",india,2019-03-02,2015,111,min,"dramas, international movies, music & musicals",a scrappy but poor boy worms his way into a ty...


## Queries

In [92]:
# Variables que tomarán el valor de los parámetros de filtro de las queries.
platform = 'hulu'
type = 'movie'
scored = 2
year = 2019
duration_type = 'min'

### Query 1

In [56]:
# Prueba los filtros antes de hacer la funcion Query 1
# Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. 
# (la función debe llamarse get_max_duration(year, platform, duration_type))
# Q1 = All[(All['platform'] == platform) & (All['release_year'] == year) & (All['duration_type'] == duration_type)].sort_values(by = 'duration_int' , ascending=False)
# Q1['title'].values[0]

"marvel studios' avengers: endgame"

In [57]:
# Funcion Query 1
# Devuelve: Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. 
def get_max_duration(year:int, platform:str, duration_type:str): 
    Q1 = All[(All['platform'] == platform) & (All['release_year'] == year) & (All['duration_type'] == duration_type)].sort_values(by = 'duration_int' , ascending=False)
    if len(Q1) > 0:
        result = Q1['title'].values[0] 
        return result
    else:
        return ("Not titles found with those parameters.")
    
get_max_duration(year, platform, duration_type)

"marvel studios' avengers: endgame"

### Query 2

In [58]:
# Prueba los filtros antes de hacer la funcion Query 2
# Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año 
# (la función debe llamarse get_score_count(platform, scored, year))
# Score[(Score['platform'] == platform) & (Score['score'] >= scored) & (All['release_year'] == year)].shape[0]

154

In [59]:
# Funcion Query 2
def get_score_count(platform, scored, year):
    Q2 = Score[(Score['platform'] == platform) & (Score['score'] >= scored) & (All['release_year'] == year)].shape[0]
    return Q2
      
get_score_count(platform, scored, year)  

154

### Query 3

In [63]:
# Funcion Query 3
# Cantidad de películas por plataforma con filtro de PLATAFORMA. (La función debe llamarse get_count_platform(platform))
def get_count_platform(platform:str, All:pd.DataFrame):
    All = All.loc[All['platform'] == platform]
    movies_platform = len(All.index)
    return movies_platform

get_count_platform(platform, All)


1450

### Query 4

In [125]:
# Actor que más se repite según plataforma y año. (La función debe llamarse get_actor(platform, year))
Not_found = {'actor': 'Not found', 'appearances': 0}
Q4_1 = All[(All['platform'] == platform) & (All['release_year'] == year)]   # Filtra por plataforma y anio
if len(Q4_1) == 0:                              # Si no hay valores para esos filtros: No encontrado
    print(Not_found)
else:    
    Q4_2 = Q4_1.assign(actor=Q4_1.cast.str.split(',')).explode('cast')  # Divide 'cast' por comas
    mask = Q4_2['actor'].isna()                 # Filtra los valores nulos
    Q4_3 = Q4_2.loc[mask, 'actor']
    if len(Q4_3) == 0:                          # Si no hay valores luego de filtrar nulos: No encontrado
        print(Not_found)
    else:
        Q4_4 = Q4_3.cast.value_counts()         # Crea una lista con actores y apariciones.
        max_actor = Q4_4.index[0]               # Obtiene el actor con más apariciones
        max_count = int(Q4_4.iloc[0])           # Obtiene el número de apariciones
        Q5 = dict({'actor': max_actor, 'appearances': max_count})
        print(Q5)
    

{'actor': 'Not found', 'appearances': 0}


In [117]:

mask = Q4_2['actor'].isna()
Q4_3 = Q4_2.loc[mask, 'actor']
len(Q4_3)

0

In [91]:
# Funcion Query 4
def get_actor(platform:str, year:int):
    All = pd.read_csv("data/all.csv")
    Q4_1 = All[(All['platform'] == platform) & (All['release_year'] == year)]   # Filtra por plataforma y anio
    if len(Q4_1) > 0: 
        Q4_2 = Q4_1.assign(actor=Q4_1.cast.str.split(',')).explode('cast')  # Divide 'cast' por comas
        Q4_3 = Q4_2.cast.value_counts()         # Crea una lista con actores y apariciones.
        max_actor = Q4_3.index[0]               # Obtiene el actor con más apariciones
        max_count = int(Q4_3.iloc[0])           # Obtiene el número de apariciones
        if max_actor == 'nan':                  # Si los valores nulos son los mas frecuentes
            max_actor = Q4_3.index[1]           # Obtiene el segundo actor con más apariciones
            max_count = int(Q4_3.iloc[1])       # Obtiene el número de apariciones correspondiente
        Q4 = dict({'actor': max_actor, 'appearances': max_count})
        return Q4
    else:
        return ({'actor': '', 'appearances': 0})
    
get_actor(platform, year)

{'actor': 'dan nachtrab', 'appearances': 3}

In [ ]:
# All = pd.read_csv("data/all.csv")
# All = All.loc[All['release_year'] == year]
# All = All.loc[All['platform'] == platform]
# All = All.loc[All['duration_type'] == duration_type]
# All['duration_int'].max()


In [ ]:
# All.to_csv('data/all.csv', index=False)